<a href="https://colab.research.google.com/github/Sinesipo/myPack/blob/master/Classification_Hackathon_1074.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Classification Hackathon

For the specifications for today's Hackathon use the slides linked below. Just note a couple things before you start:
* Use your full name and ```_EDSA``` as your Zindi username.
* The dataset for this challenge is very large and will take a long time to process. In order to use your time wisely, only use a small subset of the data to figure out how to solve this challenge, and once you're happy with that, train your model with the entire dataset.
* This Zindi challenge is tough. This will be taken into account when the supervisors mark your work. Do not worry too much about your placement on the leaderboard. In the Regression Hackathon ```laura_the_explorer``` was in first place but is outside the top 100 in this challenge
* To submit your Hackathon to Athena, zip your notebook and your submission csv file, and upload that here. Note that your report card will say you have 100% once you submit your file.
* Please attach the *Honour code* (below) cell to your notebook. 

Further instructions found on these slides: https://docs.google.com/presentation/d/1AbVndI5aOd27Jm0E1qNoYzRtWiZ6-DE3BDE0djGxzIk/edit?usp=sharing

** Good luck! **

## Honour Code
I *Sinesipo Mbasa, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [0]:
import numpy as np
import pandas as pd

In [0]:
train = pd.read_csv('/content/drive/My Drive/Fraud detection/training.csv')
test = pd.read_csv('/content/drive/My Drive/Fraud detection/test (1).csv')
df = pd.concat([train, test], axis=0, sort=True)

#df = pd.concat((train.loc[:, 'TransactionId': 'PricingStrategy'], test.loc[:,'TransactionId':'PricingStrategy']))

In [0]:
#train.head()

In [154]:
train.shape

(95662, 16)

In [155]:
test.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [156]:
test.shape

(45019, 15)

In [157]:
df.shape

(140681, 16)

In [158]:
df.head()

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value
0,AccountId_3957,1000.0,BatchId_36123,ChannelId_3,256,UGX,CustomerId_4406,0.0,2,airtime,ProductId_10,ProviderId_6,SubscriptionId_887,TransactionId_76871,2018-11-15T02:18:49Z,1000
1,AccountId_4841,-20.0,BatchId_15642,ChannelId_2,256,UGX,CustomerId_4406,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_73770,2018-11-15T02:19:08Z,20
2,AccountId_4229,500.0,BatchId_53941,ChannelId_3,256,UGX,CustomerId_4683,0.0,2,airtime,ProductId_1,ProviderId_6,SubscriptionId_222,TransactionId_26203,2018-11-15T02:44:21Z,500
3,AccountId_648,20000.0,BatchId_102363,ChannelId_3,256,UGX,CustomerId_988,0.0,2,utility_bill,ProductId_21,ProviderId_1,SubscriptionId_2185,TransactionId_380,2018-11-15T03:32:55Z,21800
4,AccountId_4841,-644.0,BatchId_38780,ChannelId_2,256,UGX,CustomerId_988,0.0,2,financial_services,ProductId_6,ProviderId_4,SubscriptionId_3829,TransactionId_28195,2018-11-15T03:34:21Z,644


first check if there are any missing values on our data sets.

In [159]:
train.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

In [160]:
test.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
dtype: int64

there is no missing values on both data sets

In [0]:
df['Year'] = pd.DatetimeIndex(df['TransactionStartTime']).year

# Drop the TransactionStartTime  column because year I just need the year
df.drop(['TransactionStartTime'], axis = 1, inplace = True)

In [0]:
#AccountId	Amount	BatchId	ChannelId	CountryCode	CurrencyCode	CustomerId	FraudResult	PricingStrategy	ProductCategory	ProductId	ProviderId	SubscriptionId	TransactionId	TransactionStartTime	Value

In [163]:
df['ProviderId'].describe

<bound method NDFrame.describe of 0        ProviderId_6
1        ProviderId_4
2        ProviderId_6
3        ProviderId_1
4        ProviderId_4
5        ProviderId_6
6        ProviderId_5
7        ProviderId_4
8        ProviderId_6
9        ProviderId_1
10       ProviderId_6
11       ProviderId_4
12       ProviderId_1
13       ProviderId_3
14       ProviderId_3
15       ProviderId_3
16       ProviderId_5
17       ProviderId_6
18       ProviderId_4
19       ProviderId_5
20       ProviderId_4
21       ProviderId_5
22       ProviderId_4
23       ProviderId_6
24       ProviderId_6
25       ProviderId_4
26       ProviderId_6
27       ProviderId_6
28       ProviderId_4
29       ProviderId_5
             ...     
44989    ProviderId_4
44990    ProviderId_3
44991    ProviderId_4
44992    ProviderId_4
44993    ProviderId_3
44994    ProviderId_4
44995    ProviderId_3
44996    ProviderId_4
44997    ProviderId_3
44998    ProviderId_3
44999    ProviderId_3
45000    ProviderId_4
45001    ProviderId_

In [164]:
df['ProductCategory'].describe

<bound method NDFrame.describe of 0                   airtime
1        financial_services
2                   airtime
3              utility_bill
4        financial_services
5                   airtime
6                   airtime
7        financial_services
8                   airtime
9        financial_services
10                  airtime
11       financial_services
12       financial_services
13       financial_services
14       financial_services
15       financial_services
16       financial_services
17             data_bundles
18       financial_services
19                  airtime
20       financial_services
21                  airtime
22       financial_services
23                  airtime
24                  airtime
25       financial_services
26                  airtime
27                       tv
28       financial_services
29       financial_services
                ...        
44989    financial_services
44990               airtime
44991    financial_services
44992         

In [0]:
#Encode ProductCategory column
categorical = ['ProductCategory', 'ProviderId']

for var in categorical:
    df=pd.concat([df, 
                   pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [166]:
df['PricingStrategy'].describe()

count    140681.000000
mean          2.257469
std           0.735089
min           0.000000
25%           2.000000
50%           2.000000
75%           2.000000
max           4.000000
Name: PricingStrategy, dtype: float64

In [0]:
#Encode PricingStrategy column
categorical = ['PricingStrategy']

for var in categorical:
    df=pd.concat([df, 
                   pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [168]:
df['BatchId'].describe()

count            140681
unique           139493
top       BatchId_67019
freq                 28
Name: BatchId, dtype: object

In [0]:
#df.drop(['BatchId', 'AccountId','SubscriptionId','CustomerId','ProviderId', 'ProductId', 'ChannelId'], axis = 1, inplace = True)

In [0]:
# Drop Columns that won't be useful and sum of these columns are redundant

df.drop(['SubscriptionId','ProductId','ChannelId','CustomerId', 'CurrencyCode', 'CountryCode', 'AccountId', 'BatchId'], axis = 1, inplace = True)

In [171]:
# total fraud and total not fraud 
df['FraudResult'].value_counts()

0.0    95469
1.0      193
Name: FraudResult, dtype: int64

In [172]:
df.head()

,Amount,FraudResult,TransactionId,Value,Year,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_retail,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ProviderId_ProviderId_1,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4
0,1000.0,0.0,TransactionId_76871,1000,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,-20.0,0.0,TransactionId_73770,20,2018,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,500.0,0.0,TransactionId_26203,500,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,20000.0,0.0,TransactionId_380,21800,2018,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0
4,-644.0,0.0,TransactionId_28195,644,2018,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [0]:
df.drop(['TransactionId'], axis = 1, inplace = True)

In [174]:
df.shape

(140681, 25)

In [0]:
# # convert to category dtype
# df['Year'] = df['Year'].astype('category')
# # convert to category codes
# df['Year'] = df['Year'].cat.codes

In [0]:
# df['Year'].tail()

In [0]:
# # convert to category dtype
# df['TransactionId'] = df['TransactionId'].astype('category')
# # convert to category codes
# df['TransactionId'] = df['TransactionId'].cat.codes

In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 25 columns):
Amount                                140681 non-null float64
FraudResult                           95662 non-null float64
TransactionId                         140681 non-null object
Value                                 140681 non-null int64
Year                                  140681 non-null int64
ProductCategory_airtime               140681 non-null uint8
ProductCategory_data_bundles          140681 non-null uint8
ProductCategory_financial_services    140681 non-null uint8
ProductCategory_movies                140681 non-null uint8
ProductCategory_other                 140681 non-null uint8
ProductCategory_retail                140681 non-null uint8
ProductCategory_ticket                140681 non-null uint8
ProductCategory_transport             140681 non-null uint8
ProductCategory_tv                    140681 non-null uint8
ProductCategory_utility_bill          140681 n

In [0]:
# categorical = ['ProductCategory']

# for var in categorical:
#     df = pd.concat([df, 
#                     pd.get_dummies(df[var], prefix=var)], axis=1)
#     del df[var]

In [0]:
# df = pd.concat([df, pd.get_dummies(df['ProductCategory'], prefix='ProductCategory')], axis=1)
# del df['ProductCategory']

In [0]:
#df.head()

#  Logistic model



Now that the data is preprocessed , all that is left is to feed our data that has been cleaned, encoded and scaled to a Logistic model

In [0]:
# Import some libraries we will need
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Train/test split


In [0]:
train_df = df[pd.notnull(df['FraudResult'])]
X_test = df[pd.isnull(df['FraudResult'])].drop(['FraudResult'], axis=1)

In [267]:
train_df.shape

(95662, 24)

In [268]:
test_df.shape

(45019, 23)

# Validation set

In [0]:
# X =train.drop(['FraudResult'], axis = 1)
# y = train['FraudResult']


In [0]:
from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(
#     train.drop(['FraudResult'], axis=1),
#     train['FraudResult'], 
#     test_size=0.2, random_state=42)

In [0]:
# for i in [X_train, X_val, X_test]:
#     print(i.shape)


Create Logistic model


In [0]:
X =train_df.drop(['FraudResult'], axis = 1)
y = train_df['FraudResult']


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

## Training

import LogisticRegression from sklearn.linear_model

In [0]:
from sklearn.linear_model import LogisticRegression

creating an object for logistic regression.

In [0]:
lm = LogisticRegression(penalty = 'l2')

In [197]:
lm.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [198]:
lm.intercept_[0]

-0.0010685073408167264

In [199]:
coeff_df = pd.DataFrame(lm.coef_.T,X.columns,columns=['Coefficient'])
coeff_df

,Coefficient
Amount,-0.000016
Value,0.000020
Year,-0.003507
ProductCategory_airtime,-1.193094
ProductCategory_data_bundles,-0.297263
ProductCategory_financial_services,0.573903
ProductCategory_movies,-0.055424
ProductCategory_other,-0.003276
ProductCategory_retail,0.000000
ProductCategory_ticket,-0.138321


## Predicting

In [0]:
pred_lm = lm.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
#confusion_matrix(y_test, pred_lm)

In [0]:
# labels = ['No Fraud', 'Fraud']

# pd.DataFrame(data=confusion_matrix(y_test, pred_lm), index=labels, columns=labels)

In [0]:
from sklearn.metrics import classification_report

In [0]:
# print('Classification Report')
# print(classification_report(y_test, pred_lm, target_names=['No Fraud', 'Fraud']))

the accuracy is not the best metric to use when evaluating imbalanced datasets as it can be very misleading.

In [234]:
from sklearn.ensemble import RandomForestClassifier

# train model
rfc = RandomForestClassifier(random_state = 42, n_estimators = 10)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
# predict on test set
#rfc_pred = rfc.predict(X_test)

In [0]:
#accuracy_score(y_test, rfc_pred)
   

In [254]:
from sklearn.metrics import classification_report

print('Classification Report')
print(classification_report(y_test, rfc_pred, target_names=['No Fraud', 'Fraud']))

Classification Report
              precision    recall  f1-score   support

    No Fraud       1.00      1.00      1.00     19089
       Fraud       0.94      0.75      0.84        44

    accuracy                           1.00     19133
   macro avg       0.97      0.87      0.92     19133
weighted avg       1.00      1.00      1.00     19133



improvement on precision, recall and on F1 score which is better prediction.

In [255]:
test.shape

(45019, 15)

##

In [0]:
#Submition = test[['TransactionId']]

In [0]:
#Submition['FraudResult'] = rfc_pred

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [277]:
for i in [X_train, X_val, test]:
    print(i.shape)

(76529, 23)
(19133, 23)
(45019, 15)


In [0]:
lm = LogisticRegression(penalty='l2')

In [279]:
lm.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred = lm.predict(test_df)

In [0]:
submition = test[['TransactionId']]

In [288]:
submition['FraudResult'] = y_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [289]:
submition

,TransactionId,FraudResult
0,TransactionId_50600,0.0
1,TransactionId_95109,0.0
2,TransactionId_47357,0.0
3,TransactionId_28185,0.0
4,TransactionId_22140,0.0
5,TransactionId_134338,0.0
6,TransactionId_109096,0.0
7,TransactionId_14249,0.0
8,TransactionId_69896,0.0
9,TransactionId_91468,0.0


In [296]:
# Fiting the rf model
rf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
# Predict the y values
y_pred = rf.predict(test_df)

In [0]:
Submition1 = test[['TransactionId']]

In [300]:
Submition1['FraaudResult'] = y_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
Submition1.to_csv('Submition1.csv', index = False)

In [303]:
Submition1.head()

,TransactionId,FraaudResult
0,TransactionId_50600,0.0
1,TransactionId_95109,0.0
2,TransactionId_47357,0.0
3,TransactionId_28185,0.0
4,TransactionId_22140,0.0
